# Recomendação de Produtos Utilizando NLP:

O objetivo desse estudo é criar recomendações de produtos similiares ou complementares baseado na descrição dos produtos

In [37]:
import pandas as pd

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Carregando Dataset:

In [38]:
df = pd.read_excel("../data/online_retail_cleaned.xlsx")

In [39]:
# Removendo descrições duplicadas
products_df = (
    df[["stock_code_cleaned", "description_cleaned"]]
    .drop_duplicates()
    .reset_index(drop=True)
)

# TF-IDF (Term Frequency – Inverse Document Frequency):

Essa biblioteca foi utilizada para medir a importância de algumas palavras em relação a todas as outras.

Palavras muito comuns recebem peso baixo, palavras mais específicas recebem peso alto.

Depois, transforma tudo isso em um número, armazenando em 'tfidf_matrix'.

***O objetivo é facilitar a análise com NLP.***

In [40]:
vectorizer = TfidfVectorizer(
    stop_words="english", # remove palavras obsoletas ('the', 'and', 'or'...)
    ngram_range=(1,3), # analisa palavras únicas e compostas (2 e 3 palavras)
)

tfidf_matrix = vectorizer.fit_transform(
    products_df["description_cleaned"]
)

# Similaridade do Cosseno:

A **similaridade do cosseno** mede o quanto dois vetores apontam na **mesma direção**, independentemente do seu tamanho (magnitude).

### Definição matemática

Dado dois vetores $[\mathbf{A}]$ e $[\mathbf{B}]$, a similaridade do cosseno é definida como:

$
\text{cosine\_similarity}(\mathbf{A}, \mathbf{B}) =
\dfrac{\mathbf{A} \cdot \mathbf{B}}
{\|\mathbf{A}\| \, \|\mathbf{B}\|}
$


$
0 \leq \text{cosine\_similarity} \leq 1
$

- $[1.0]$ → textos **idênticos ou quase idênticos**
- $[\approx 0.7]$ → textos **bastante semelhantes**
- $[\approx 0.0]$ → textos **sem relação semântica**

> Os vetores TF-IDF nunca são negativos.

- Vetores próximos → **ângulo pequeno** → alta similaridade
- Vetores distantes → **ângulo grande** → baixa similaridade

In [41]:
# Atribui valor de similaridade para cada descrição
similarity_matrix = cosine_similarity(tfidf_matrix)

In [42]:
def recomendar_produtos_similares(
    stock_code,
    products_df,
    similarity_matrix,
    top_n=5
):
    # Encontrar o índice do produto
    idx = products_df.index[
        products_df["stock_code_cleaned"] == stock_code
    ][0]

    # Similaridade do produto com todos os outros
    similarities = similarity_matrix[idx]

    # Criar um Series (índice = produto, valor = similaridade)
    similarity_series = pd.Series(
        similarities,
        index=products_df.index
    )

    # Remover o próprio produto
    similarity_series = similarity_series.drop(idx)

    # Selecionar os top N mais similares
    top_indices = similarity_series.nlargest(top_n).index

    # Retornar os produtos recomendados
    return products_df.loc[top_indices]


# Exemplos:

### Produto 1:

"party invites balloon girl"

In [43]:
example_product = products_df["stock_code_cleaned"].iloc[1]
products_df[products_df["stock_code_cleaned"]==example_product]["description_cleaned"]

1    party invites balloon girl
Name: description_cleaned, dtype: object

##### Recomendações para:

"party invites balloon girl"

In [44]:
recomendar_produtos_similares(
    example_product,
    products_df,
    similarity_matrix
)

,stock_code_cleaned,description_cleaned
0,21058,party invites woodland
2,21061,party invites football
785,21059,party invites dinosaurs
3,21062,party invites spaceman
1599,21219,set/4 badges balloon girl


### Produto 2:

"15cm christmas glass ball 20 lights"

In [45]:
example_product = products_df["stock_code_cleaned"].iloc[1500]
products_df[products_df["stock_code_cleaned"]==example_product]["description_cleaned"]

1500    15cm christmas glass ball 20 lights
Name: description_cleaned, dtype: object

##### Recomendações para:

"15cm christmas glass ball 20 lights"

In [46]:
recomendar_produtos_similares(
    example_product,
    products_df,
    similarity_matrix
)

,stock_code_cleaned,description_cleaned
3316,23025,drawer knob vintage glass ball
672,20668,disco ball christmas decoration
145,22909,set of 20 vintage christmas napkins
317,79321,chilli lights
421,84763,zinc finish 15cm planter pots


### Produto 3:

"doormat keep calm and come in"

In [47]:
example_product = products_df["stock_code_cleaned"].iloc[3000]
products_df[products_df["stock_code_cleaned"]==example_product]["description_cleaned"]

3000    doormat keep calm and come in
Name: description_cleaned, dtype: object

##### Recomendações para:

"doormat keep calm and come in"

In [48]:
recomendar_produtos_similares(
    example_product,
    products_df,
    similarity_matrix
)

,stock_code_cleaned,description_cleaned
649,48194,doormat hearts
3650,23543,keep calm wall art
3670,23543,wall art keep calm
3786,23691,keep calm birthday wrap
3788,23691,wrap keep calm birthday
